<span type="title">Spring IOC</span> | <span type="update">2018-09-30</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">这一部分主要介绍使用 Spring 框架进行 Web 开发简化的方法。本章重点介绍了 Spring 的 IOC/DI 控制反转/依赖注入，包括使用 XML 和 注解两种以无侵入性的方式创建对象实例的方法。IOC 是工厂方法的升级版本，其优点在于：使用 XML，依赖 Spring 提供的IOC容器，对象可以更加方便的在运行时动态生成，而不需手动构造（可以将 Spring IOC 看作一个大的对象工厂，其根据 XML 进行 RTTI 参数注入，从容器中根据 id 获取对象。）。</p></span>

# Spring IOC 概要

## IOC 使用方法一览 - XML 方式

Spring 的 IOC 需要使用以下 jar 依赖：commons-logging, core, beans, context, expression.

其基本使用方法如下：

person Bean 类

```java
public class Person {
    private String name;
    private int id;
    private Address address;
    private List<Address> schools;
... 略}
```

IOC 容器 Bean 配置文件

```xml
<bean id="ccnu" class="com.mazhangjing.test.Address" scope="singleton">
    <constructor-arg index="0" value="China"/>
    <constructor-arg name="city" value="Wuhan" />
    <constructor-arg name="code" value="027" />
    <constructor-arg index="3" value="CCNU" />
</bean>
<bean id="person" class="com.mazhangjing.test.Person" scope="singleton">
    <property name="name" value="Corkine"/>
    <property name="address" ref="ccnu" />
    <property name="id">
        <value>2017110658</value>
    </property>
</bean>
```

IOC 对象调用

```java
ApplicationContext ctx = new ClassPathXmlApplicationContext("applicationContext.xml");
Person person = (Person) ctx.getBean("person");
```

通过在 context.xml 进行配置，然后 IOC 容器会自动解析此 XML 文件，并且进行 Bean 的构造，并且自动管理其生命周期。当需要使用 Bean 的时候，从 ApplicationContext 中根据 id 获取对象即可使用。这就是 IOC 容器的基本使用方法。重点在于 XML 配置 Bean 以及其依赖关系、各种数据结构、外部属性文件的语法。

## IOC/DI 的定义

IOC 指的是 Inversion of Control。在传统的 JavaEE 中，当希望获取一个对象，我们要向容器发送请求，获取构造此对象的资源，容器进行相应，然后构造此对象。在IOC中，容器主动的在项目启动的时候，自动将合适的资源注入对象中，这样，只需要向容器寻求这个对象即可，而不用再去调用和配置对象构造器来手动管理对象构建。IOC和DI含义相同，其中DI指的是依赖注入，也就是容器将对象的类属性所需要的对象使用 setter 方法进行自动调用，然后既可以直接使用这个对象，而不需手动的去 set 依赖的方式。DI是IOC的另一种表述形式。

## IOC/DI 的前世今生

IOC 产生的前世今生是这样的：我们现在有很多对象，这些对象需要被协调用来做某些事情，为了对象和对象间的互相通信，我们一般会在一个对象内保留其余其可能与之交互对象的实例，实例通过构造器传入或者通过 setter 方法进行设置，进行设置的过程称之为对象的装配。

不难发现，对象的装配是一件复杂的事情，因为，我们在进行某项工作的时候，为了得到与之交互的对象，不能总是就地新建一堆依赖的对象实例，然后进行对象的装配和使用。为了做到对象和对象的兼容以及协调，我们必须在对象内保存其余对象的实例引用，因此就必须有一个中央提供其所需的依赖组件，为其提供实例引用。其次，如果我们真的就地创建实例引用，那么这些创建此对象的代码将难以被其余需要相同依赖的其它对象进行重用。因此，我们必须在某个地方集中创建类实例，然后分配这些实例引用给对象进行装配。第一次革命是解决这个问题而产生的，革命的结果是：抽象工厂。

抽象工厂显著的降低了对象装配的负担。对象只需要从上下文获取工厂的一个引用即可获取工厂提供的对象实例引用，从而用来进行对象组装。对象甚至不需要知道工厂实际初始化的是什么类型的对象，以及为其接口提供服务的对象是从哪里来的。尽管拥有这些好处，但是抽象工厂亦有其缺点。抽象工厂的问题在于，其接口限制了从其中调用对象类型的灵活性。

IOC/DI就是为了解决此问题而生的：在DI中，对象甚至不用去得到一个抽象工厂的引用，然后调用工厂获得实例引用。容器会自动的为缺少依赖的对象注入依赖所需要的“单例”对象引用，这样对象就可以直接拿来使用了。当然，为了做到这一点，需要对这些作为组件的对象添加注解，为这些需要依赖的对象添加注入。

# 自动化装配

Spring 4 以后，推荐使用自动化装配方案，这种方案避免了 XML 设置的冗长，同时对于 Java 也没有过多的侵入性（只是添加了一些注解而已，并且这些注解很多还是JavaEE兼容的）。自动化装配技术关键有二：component scanning 组件扫描以及 autowiring 自动装配。组件扫描是为了将依赖搜集整理，然后在容器中进行 Bean 的构造， autowiring 自动装配则是自动将这些依赖进行对象的注入。

